# Initial training

In [42]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd



path = 'smoke_detection_iot.csv'

data = pd.read_csv(path).drop(['Unnamed: 0', 'UTC', 'CNT'], axis=1)
Y_df = data['Fire Alarm']
X_df = data.drop('Fire Alarm', axis=1)

Y_test_5f = X_df.values
Y_pred = Y_df.values

X_train_raw, X_test_raw, Y_train_5f, Y_test_5f = train_test_split(Y_test_5f, Y_pred, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_raw = scaler.fit_transform(X_train_raw)
X_test_raw = scaler.transform(X_test_raw)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from alarmnetclass import AlarmNet
import torch
from torch import nn

alpha = 1e-2
epochs = 3000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AlarmNet(
    num_features=X_train_raw.shape[1],
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)

X_train_raw_device = torch.tensor(X_train_raw).float().to(device)
X_test_raw_device = torch.tensor(X_test_raw).float().to(device)
Y_train_raw_device = torch.tensor(Y_train_5f).float().view(-1, 1).to(device)
Y_test_raw_device = torch.tensor(Y_test_5f).float().view(-1, 1).to(device)
model.train(epochs, X_train_raw_device, X_test_raw_device, Y_train_raw_device, Y_test_raw_device, alpha)
model.print_results()

Epoch 0 Loss: 0.6690529584884644
Epoch 500 Loss: 0.003392779966816306
Epoch 1000 Loss: 0.0006458048592321575
Epoch 1500 Loss: 0.00026241334853693843
Epoch 2000 Loss: 0.00015851714124437422
Epoch 2500 Loss: 0.00010259856207994744
Accuracy: 0.9995209963276385
Precision: 0.9995517202734506
Recall: 0.9997758098867839
F1: 0.9996637525218561
Confusion_matrix:
[[3601    4]
 [   2 8919]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3605
         1.0       1.00      1.00      1.00      8921

    accuracy                           1.00     12526
   macro avg       1.00      1.00      1.00     12526
weighted avg       1.00      1.00      1.00     12526



# Feature Analysis
- Initially there are 15 features
- 3 are unusable
    - UTC Timestamp
    - CNT
    - Unnamed: 0
- 12 features are usable

In [44]:
import numpy as np



corr = np.abs(data.corr())
ranking = corr['Fire Alarm'].sort_values(ascending=False)[1:]
print(ranking)

Humidity[%]       0.399846
Raw Ethanol       0.340652
Pressure[hPa]     0.249797
TVOC[ppb]         0.214743
Temperature[C]    0.163902
NC0.5             0.128118
PM1.0             0.110552
Raw H2            0.107007
eCO2[ppm]         0.097006
PM2.5             0.084916
NC1.0             0.082828
NC2.5             0.057707
Name: Fire Alarm, dtype: float64


In [45]:
num_drops = 8
remaining_features = ranking.index[:-num_drops]


X_df_dropped = data[remaining_features]
print('Remaining features:', X_df_dropped.columns)
X_train_dropped, X_test_dropped, Y_train_dropped, Y_test_dropped = train_test_split(X_df_dropped.values, Y_pred, test_size=0.2, random_state=0)

X_train_dropped = scaler.fit_transform(X_train_dropped)
X_test_dropped = scaler.transform(X_test_dropped)


Remaining features: Index(['Humidity[%]', 'Raw Ethanol', 'Pressure[hPa]', 'TVOC[ppb]'], dtype='object')


In [46]:
dropped_model = AlarmNet(
    num_features=X_train_dropped.shape[1],
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)

X_train_dropped_device = torch.tensor(X_train_dropped).float().to(device)
X_test_dropped_device = torch.tensor(X_test_dropped).float().to(device)
Y_train_dropped_device = torch.tensor(Y_train_dropped).float().view(-1, 1).to(device)
Y_test_dropped_device = torch.tensor(Y_test_dropped).float().view(-1, 1).to(device)

dropped_model.train(epochs, X_train_dropped_device, X_test_dropped_device, Y_train_dropped_device, Y_test_dropped_device, alpha)
dropped_model.print_results()

Epoch 0 Loss: 0.7509467601776123
Epoch 500 Loss: 0.015842126682400703
Epoch 1000 Loss: 0.06398574262857437
Epoch 1500 Loss: 0.050980836153030396
Epoch 2000 Loss: 0.04781293869018555
Epoch 2500 Loss: 0.046131569892168045
Accuracy: 0.999041992655277
Precision: 0.9987683350128765
Recall: 0.999887904943392
F1: 0.9993278064082456
Confusion_matrix:
[[3594   11]
 [   1 8920]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3605
         1.0       1.00      1.00      1.00      8921

    accuracy                           1.00     12526
   macro avg       1.00      1.00      1.00     12526
weighted avg       1.00      1.00      1.00     12526



In [47]:
AlarmNet.compare_results(dropped_model.get_results(), model.get_results())

Comparing results:
accuracy: -0.0479463001438323 %
precision: -0.07843513186308494 %
recall: 0.01121076233184124 %
f1: -0.033617208633270085 %


## Feature Analysis Results
- We can remove the bottom 8 features and have a model that only loses 0.1% precision, and even less for every other metric
- A 4 feature model is almost perfect

# Error Handling
- The 12 initial features came from 4 sensors
    - Temp/Humidity
    - Pressure
    - Volatile Organic Compounds (CO2, Ethanol, H2, TVOC)
    - Particulate Matter (PM1, PM2.5, NC0.5, NC1, NC2.5)
- 3 of these sensors are redundant, with the exception being the PM sensor
    - This means that the features related to the PM sensor are twice as likely to be missing
- We can simulate a real world scenario by introducing error according to this distribution



In [48]:
import random
from sklearn.impute import SimpleImputer

# Add back the most correlated PM feature, so that all 4 sensors are used
# PM0.5 is the most correlated PM feature, with index 5
remaining_features_2 = list(remaining_features)
remaining_features_2.append(ranking.index[5])
X_error = data[remaining_features_2]
# For each measurement, each sensor has this chance of introducing an error
error_chance = 0.2

# Introduce sensor errors

VOC_features = [
    'TVOC[ppb]',
    'eCO2[ppm]',
    'Raw H2',
    'Raw Ethanol'
]

PM_features = [
    'PM1.0',
    'PM2.5',
    'NC0.5',
    'NC1.0',
    'NC2.5'
]

th_features = [
    'Temperature[C]',
    'Humidity[%]'
]
pressure_features = [
    'Pressure[hPa]'
]

# The PM sensor is twice as likely to fail due to lack of redundancy
chances = [error_chance, error_chance*2, error_chance, error_chance]
sensors = [VOC_features, PM_features, th_features, pressure_features]
for datapoint in X_error.values:
    errored_features = []
    for i, sensor in enumerate(sensors):
        sensor_error = random.random() < chances[i]
        if sensor_error:
            errored_features.extend(sensor)
    errored_features = [feature for feature in errored_features if feature in X_error.columns]
    for feature in errored_features:
        datapoint[X_error.columns.get_loc(feature)] = np.nan

X_train_error, X_test_error, Y_train_error, Y_test_error = train_test_split(X_error, Y_pred, test_size=0.2, random_state=0)

## Imputation
- Replace each errored value with the mean of that feature from the training data

In [49]:
imputer = SimpleImputer(strategy='mean')
X_impute_train = imputer.fit_transform(X_train_error)
X_impute_test = imputer.transform(X_test_error)

X_impute_train = scaler.fit_transform(X_impute_train)
X_impute_test = scaler.transform(X_impute_test)

In [50]:
imputed_model = AlarmNet(
    num_features=X_impute_train.shape[1],
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)

X_impute_train_device = torch.tensor(X_impute_train).float().to(device)
X_impute_test_device = torch.tensor(X_impute_test).float().to(device)
Y_train_impute_device = torch.tensor(Y_train_error).float().view(-1, 1).to(device)
Y_test_impute_device = torch.tensor(Y_test_error).float().view(-1, 1).to(device)

imputed_model.train(
    epochs,
    X_impute_train_device,
    X_impute_test_device,
    Y_train_impute_device,
    Y_test_impute_device,
    alpha,
    loss_fn = nn.MSELoss()
)
imputed_model.print_results()
AlarmNet.compare_results(imputed_model.get_results(), dropped_model.get_results())

Epoch 0 Loss: 0.23539188504219055
Epoch 500 Loss: 0.003432096913456917
Epoch 1000 Loss: 0.0012487940257415175
Epoch 1500 Loss: 0.0005852619069628417
Epoch 2000 Loss: 0.0003480861778371036
Epoch 2500 Loss: 0.00023499401868321002
Accuracy: 0.9999201660546064
Precision: 1.0
Recall: 0.999887904943392
F1: 0.9999439493301945
Confusion_matrix:
[[3605    0]
 [   1 8920]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3605
         1.0       1.00      1.00      1.00      8921

    accuracy                           1.00     12526
   macro avg       1.00      1.00      1.00     12526
weighted avg       1.00      1.00      1.00     12526

Comparing results:
accuracy: 0.08782435129740421 %
precision: 0.12316649871234597 %
recall: 0.0 %
f1: 0.061617745910819435 %


## Error Modes
- Current features:
    - Humidity
    - Raw Ethanol
    - Pressure
    - TVOC
    - NC0.5
- Sensors:
    - Humidity
    - Pressure
    - Raw Ethanol/TVOC
    - NC0.5
- The model should be able to handle missing data in the case where at most 3 sensors have failed, because 4 failed sensors means no data
    - 1 failed sensor = 4c1 = 4
    - 2 failed sensors = 4c2 = 6
    - 3 failed sensors = 4c3 = 4
    - Total = 14
## Ensemble Training
- We can train 14 models that can predict the missing data for each error mode
- In the case of error, we select the model that corresponds to the error mode and use it to predict the missing data
- Then use the main model to predict the target
### Indexing Ensemble
- Columns should be rearranged according to sensors
    - Humidity, Pressure, NC0.5, Ethanol, TVOC, 
- The error mode can be represented as a 4-bit value
    - 0b0000 = No error
    - 0b0001 = Ethanol/TVOC Error
    - 0b0010 = NC0.5 Error
    - 0b0100 = Pressure error
    - 0b1000 = Humidity error
- A 5-bit value can represent which features are missing
    - 0b00000 = No error
    - 0b00011 = Ethanol/TVOC Error
    - 0b00100 = NC0.5 Error
    - 0b01000 = Pressure error
    - 0b10000 = Humidity error
- We can convert from the 5-bit value to the 4-bit value with a simple shift right operation

### Model Table
- Store the models with an array
- The index of the model is the error mode
- Model 15 will always predict 1, because if all sensors have failed the worst should be assumed for safety
- Model 0 will be the standard model trained on a full dataset
- The rest of the arrays will be trained on the data with the corresponding error mode
### New Model Type
- A new model class will be created that will predict the missing values, construct the repaired dataset, call the standard model, and return the result
### Ensemble Class
- This new class will hold the model table and the standard model
- It will be responsible for constructing the model address, calling the correct model, and returning the result



    



In [51]:
class ConstantPredictor(AlarmNet):
    def __init__(self, val):
        self.val = val
    def predict(self, x):
        return torch.tensor([self.val]*x.shape[0]).reshape(-1, 1).float()
    def train(self, *args, **kwargs):
        pass
    def get_results(self):
        return {
            'accuracy': None,
            'precision': None,
            'recall': None,
            'f1': None,
            'confusion_matrix': None,
            'classification_report': None
        }
    def print_results(self):
        super().print_results(self.get_results())
    

In [52]:
from sklearn.metrics import r2_score

class DataPredictor(nn.Module):
    @classmethod
    def compare_results(cls, results1, results2):
        print(100 * (results1 - results2) / results1)
        
    def __init__(self, error_mode, hidden_layers=[64,32], activation=nn.ReLU,):
        super().__init__()
        self.error_mode = error_mode
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.input_cols = []
        self.output_cols = []
        
        #Error Mode is a 5 bit integer, with each bit representing a feature
        # If the bit is 1, the feature is errored
        output_features = 0
        input_features = 0
        for i in range(5):
            if error_mode & (1 << i):
                output_features += 1
                self.output_cols.append(4-i)
            else:
                self.input_cols.append(4-i)
        input_features = 5-output_features
        

        
        self.stack_list = [nn.Linear(input_features, hidden_layers[0]), activation()]
        for i in range(1, len(hidden_layers)):
            self.stack_list.extend([nn.Linear(hidden_layers[i-1], hidden_layers[i]), activation()])
        self.stack_list.extend([nn.Linear(hidden_layers[-1], output_features)])
        self.stack = nn.Sequential(*self.stack_list)
        
        
        
    def train(self, epochs, X_train, X_test, alpha, loss_fn=nn.MSELoss(), Y_tr=None, Y_te=None,):

        X_train_new = X_train[:, self.input_cols]
        X_test_new = X_test[:, self.input_cols]
        Y_train = X_train[:, self.output_cols]
        Y_test = X_test[:, self.output_cols]
        
        optimizer = torch.optim.Adam(self.parameters(), lr=alpha)
        for _ in range(epochs):
            optimizer.zero_grad()
            Y_pred = self.forward(X_train_new)
            loss = loss_fn(Y_pred, Y_train)
            loss.backward()
            optimizer.step()
        self.last_test = Y_test
        self.last_pred = self.forward(X_test_new)
        self.last_score = self.get_results(Y_test, self.last_pred)
        
    def forward(self, x):
        return self.stack(x)
    
    def get_results(self, Y_test=None, Y_pred=None):
        if Y_test is None:
            Y_test = self.last_test
        if Y_pred is None:
            Y_pred = self.last_pred
        Y_test = Y_test.cpu().detach().numpy()
        Y_pred = Y_pred.cpu().detach().numpy()
        
        self.last_score = r2_score(Y_test, Y_pred)
        return self.last_score
    def print_results(self):
        if self.last_score is None:
            self.get_results()
        print('R2 score:', self.last_score)


    

#### Testing with training for error mode 00011

In [53]:
new_col_order = ['Humidity[%]', 'Pressure[hPa]', 'NC0.5', 'Raw Ethanol', 'TVOC[ppb]']
new_data_drop = data[new_col_order]
#rearrange so that order is humidity, pressure, NC0.5, Ethanol, TVOC

X_train_5f, X_test_5f, Y_train_5f, Y_test_5f = train_test_split(new_data_drop, Y_pred, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_5f = scaler.fit_transform(X_train_5f)
X_test_5f = scaler.transform(X_test_5f)
layers = [64,32]
predictor_3 = DataPredictor(0b00011, hidden_layers=layers).to(device)

X_train_5f_device = torch.tensor(X_train_5f).float().to(device)
X_test_5f_device = torch.tensor(X_test_5f).float().to(device)
Y_train_5f_device = torch.tensor(Y_train_5f).float().view(-1, 1).to(device)
Y_test_5f_device = torch.tensor(Y_test_5f).float().view(-1, 1).to(device)
predictor_3.train(epochs, X_train_5f_device, X_test_5f_device, alpha)
predictor_3.print_results()

R2 score: 0.9666537046432495


In [54]:
class Error_Predictor(AlarmNet):
    def __init__(self, error_mode, hidden_layers=[64,32], activation=nn.ReLU, std_model: AlarmNet = None):
        super().__init__(
            pass_through=True
        )
        self.error_mode = error_mode
        self.std_model = std_model
        self.predictor = DataPredictor(error_mode, hidden_layers=hidden_layers, activation=activation)
    def train(self, epochs, X_train, X_test, Y_train, Y_test, alpha):
        self.predictor.train(epochs, X_train, X_test, alpha)
        self.last_pred = self.predict(X_test)
        self.last_test = Y_test
    def predict(self, X):
        X_in = X[:, self.predictor.input_cols]
        X_out = self.predictor.forward(X_in)
        #Insert the predicted values into the original tensor
        X[:, self.predictor.output_cols] = X_out
        return self.std_model.predict(X)
    def get_results(self, Y_test=None, Y_pred=None):
        if Y_test is None:
            Y_test = self.last_test
        if Y_pred is None:
            Y_pred = self.last_pred
        return self.std_model.get_results(Y_test, Y_pred)
    
        
        
        
        
        
        
    

In [61]:
std_model = AlarmNet(
    num_features=X_train_5f.shape[1],  # Update to match the new input dimensions
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)
epochs = 3000
alpha = 1e-2

X_train_5f_device = torch.tensor(X_train_5f).float().to(device)
X_test_5f_device = torch.tensor(X_test_5f).float().to(device)
Y_train_5f_device = torch.tensor(Y_train_5f).float().view(-1, 1).to(device)
Y_test_5f_device = torch.tensor(Y_test_5f).float().view(-1, 1).to(device)

std_model.train(epochs, X_train_5f_device, X_test_5f_device, Y_train_5f_device, Y_test_5f_device, alpha)

std_model.print_results()

Epoch 0 Loss: 0.699371874332428
Epoch 500 Loss: 0.011745047755539417
Epoch 1000 Loss: 0.06066644936800003
Epoch 1500 Loss: 0.05417398363351822
Epoch 2000 Loss: 0.05181897431612015
Epoch 2500 Loss: 0.05028727650642395
Accuracy: 0.9989621587098835
Precision: 0.9986565158978952
Recall: 0.999887904943392
F1: 0.999271831064807
Confusion_matrix:
[[3593   12]
 [   1 8920]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3605
         1.0       1.00      1.00      1.00      8921

    accuracy                           1.00     12526
   macro avg       1.00      1.00      1.00     12526
weighted avg       1.00      1.00      1.00     12526



In [63]:
layers = [64,64]
error_pred_3 = Error_Predictor(0b00011, hidden_layers=layers, std_model=std_model).to(device)
error_pred_3.train(epochs, X_train_5f_device, X_test_5f_device, Y_train_5f_device, Y_test_5f_device, alpha)

In [64]:
Y_pred = error_pred_3.predict(X_test_5f_device)
results_1 = error_pred_3.get_results(Y_test_5f_device, Y_pred)
error_pred_3.print_results()

imputer = SimpleImputer(strategy='mean')
X_impute_test_1 = X_test_5f.copy()


imputer.fit(X_impute_test_1)
X_impute_test_1[:, error_pred_3.predictor.output_cols] = np.nan
X_impute_test_1 = imputer.transform(X_impute_test_1)
X_impute_test_1_device = torch.tensor(X_impute_test_1).float().to(device)


Y_pred_std = std_model.predict(X_impute_test_1_device)
results_2 = std_model.get_results(Y_test_impute_device, Y_pred_std)
std_model.print_results()

AlarmNet.compare_results(results_1, results_2)

Accuracy: 0.9522593006546384
Precision: 0.9583654587509638
Recall: 0.9753390875462392
F1: 0.9667777777777777
Confusion_matrix:
[[3227  378]
 [ 220 8701]]
Classification_report:               precision    recall  f1-score   support

         0.0       0.94      0.90      0.92      3605
         1.0       0.96      0.98      0.97      8921

    accuracy                           0.95     12526
   macro avg       0.95      0.94      0.94     12526
weighted avg       0.95      0.95      0.95     12526

Accuracy: 0.8855181223056043
Precision: 0.8615860137158312
Recall: 0.999887904943392
F1: 0.9255992528795268
Confusion_matrix:
[[2172 1433]
 [   1 8920]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      0.60      0.75      3605
         1.0       0.86      1.00      0.93      8921

    accuracy                           0.89     12526
   macro avg       0.93      0.80      0.84     12526
weighted avg       0.90      0.89      0.88     

### Testing Error Predictor with 2 errors
-The predictor is much more likely to predict a false negative than the imputer, making this not the right choice for a fire alarm
- Hopefully this changes in the case of the full ensemble
- Compared to the imputer method, the predictor is better in all metrics except for recall